<a href="https://colab.research.google.com/github/JDM-1609/Statistical-Process-Control-in-Injection-Machines/blob/main/Visualization_and_Control_Charts_SPC_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VISUALIZACIÓN DE DATOS Y GRÁFICAS DE CONTROL MEDIANTE SPC

El siguiente programa recibe los archivos generados apartir del sistema ALS, separando el DataFrame en diferentes secciones especificadas (Info. Contextual, estadísticos y datos crudos) creando un DataFrame para cada una, permitiendo extraer datos de interés para llevar seguimietos de la optimización de los límites de control de los planes de control de las inyectoras en la empresa SIMEX. Con lo anterior, se logra contruir las gráficas de control (I, X̄ y S) para la identificación de mediciones fuera de especificaciones.

###TASKS
1. Se tiene ya los datos necesarios en df_resumen para completar la plantilla de seguimientos por máquina. En este punto, evaluar opción de incluir las columnas Cpk estimado y Rango ajustado, de forma tal que con un input se pueda ingresar el valor del rango ajustado al que se quiere llegar y calcule el Cpk estimado. Adicionalmente, evaluar la opción de generar un archivo PDF donde se muestre esta tabla final (Tal cual a la de la plantilla de seguimientos) y en la cabezara se muestra la informacion contextual (Pedido, Molde, Descripcion, Material, fecha de analisis, fecha datos analizados) y ademas, crea un input donde se pueda escribir comentarios y se muestren en la parte inferior del PDF



In [ ]:
import csv
from pathlib import Path
from io import StringIO
from ipywidgets import IntSlider, interact, Dropdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# OPCIONAL: para visualización de los datos en el DF

#pd.set_option("display.max_columns", 20) # Controla cuántas columnas se muestran sin omisiones
pd.set_option("display.float_format", lambda v: f"{v:.5g}") # Muestra valores con hasta 4 cifras significativas
ENCODING_ALS = "CP1250" # Encoding de los archivos del ALS

In [ ]:
# Carga de datos
from google.colab import files

uploaded = files.upload()
RUTA_IN = next(iter(uploaded.keys()))
print("Archivo cargado:", RUTA_IN)

In [ ]:
# Inicialización de los índices start & end de cada sección
                       # SECCIONES
idx_info = 0           # Información contextual
idx_stats_start = 2    # Start Estadísticos
idx_stats_end = 21     # End Estadísticos
idx_raw_start = 23     # Datos crudos

EJECUCIÓN

In [ ]:
# EJECUCIÓN

# Leer todas las líneas del archivo
lines = leer_lineas(RUTA_IN, encoding=ENCODING_ALS)
print(f"NÚMERO TOTAL DE LÍNEAS EN EL ARCHIVO: {len(lines)}\n")

# Información contextual
df_info = parse_info_contextual(lines, idx_info=idx_info)
print("INFORMACIÓN CONTEXTUAL:")
display(df_info)

# Estadísticos
df_stats = parse_estadisticos(lines, idx_start=idx_stats_start, idx_end=idx_stats_end)
print("\nTABLA ESTADÍSTICOS PRINCIPALES (Vista rápida):")
display(df_stats.head())

# Datos crudos
df_raw = parse_datos_crudos(lines, idx_start=idx_raw_start)
print("\nDATOS CRUDOS MEDIDOS POR EL ALS (Vista rápida):")
display(df_raw.head())


In [ ]:
# EJECUCIÓN
df_resumen = estadisticos_resumen(df_stats)
df_resumen

In [ ]:
# EJECUCIÓN
df_limpio, df_xbar, df_s, xbar_prom, s_prom = procesar_datos_spc(df_raw)


In [ ]:
# EJECUCIÓN: para graficar la carta I de la variable seleccionada en el desplegable
selector_carta_i(df_limpio, df_stats, window=20)


In [ ]:
# EJECUCIÓN: para graficar la carta X de la variable seleccionada en el desplegable
selector_carta_x(df_xbar, xbar_prom, df_stats, window=30)


In [ ]:
# EJECUCIÓN: para graficar la carta S de la variable seleccionada en el desplegable
selector_carta_s(df_s, s_prom, df_stats, window=30)


In [ ]:
# EJECUCIÓN: para graficar la carta I de cada variable independiente
graficar_carta_i(df_limpio, df_stats, "t4012 [s]")
graficar_carta_i(df_limpio, df_stats, "t4018 [s]")
graficar_carta_i(df_limpio, df_stats, "t4015 [s]")
graficar_carta_i(df_limpio, df_stats, "V4062 [cmł]")
graficar_carta_i(df_limpio, df_stats, "p4072 [bar]")
graficar_carta_i(df_limpio, df_stats, "V4065 [cmł]")



In [ ]:
# EJECUCIÓN: para graficar la carta X de cada variable independiente
graficar_carta_x(df_xbar, xbar_prom, df_stats, "t4012 [s]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "t4018 [s]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "t4015 [s]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "V4062 [cmł]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "p4072 [bar]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "V4065 [cmł]")

In [ ]:
# EJECUCIÓN: para graficar la carta S de cada variable independiente
graficar_carta_s(df_s, s_prom, df_stats, "t4012 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "t4012 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "t4018 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "t4015 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "V4062 [cmł]")
graficar_carta_s(df_s, s_prom, df_stats, "p4072 [bar]")
graficar_carta_s(df_s, s_prom, df_stats, "V4065 [cmł]")


FUNCIONES

In [ ]:
# FUNCIONES: División de las secciones en un DF diferente cada uno.

# Lee el CSV como strings y devuelve una lista donde cada elemento es una línea del CSV completo
def leer_lineas(ruta_csv: str, encoding: str = ENCODING_ALS):
    raw = Path(ruta_csv).read_text(encoding=encoding, errors="replace")
    lines = raw.splitlines()  # Este método siempre devuelve una lista
    return lines

# Devuelve la sección "Información Contextual" como un DF
def parse_info_contextual(lines, idx_info: int = 0) -> pd.DataFrame:
    line = lines[idx_info]  # Se toma únicamente la primera línea
    cells = next(csv.reader([line], delimiter=",", quotechar='"'))

    # Limpieza eliminando vacíos y el elemento "Valores reales"
    cleaned = []
    for c in cells:
        c = c.strip().strip('"')
        if not c:  # Si está vacío -> se omite
            continue
        if c.lower() == "valores reales":
            continue
        cleaned.append(c)

    claves  = cleaned[0::2]
    valores = cleaned[1::2]

    df_info = pd.DataFrame([valores], columns=claves)
    return df_info

# Devuelve la sección "Estadísticos" como un DF
def parse_estadisticos(lines, idx_start: int, idx_end: int) -> pd.DataFrame:

    # Se unen las líneas de la sección en un solo string tipo CSV
    block = "\n".join(lines[idx_start:idx_end])

    # Leemos este bloque como si fuera un CSV independiente
    df_stats = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
        )
    # Los índices del df son los nombres de los estadísticos
    df_stats.index.name = "Estadísticos"

    # Ajuste de los encabezados de las columnas
    cols = list(df_stats.columns)
    if str(cols[0]).strip() == "":
      df_stats.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izq
      df_stats = df_stats.dropna(axis=1, how="all") # Elimina la columna vacía

    return df_stats

# Devuelve la sección "Datos Crudos" como un DF
def parse_datos_crudos(lines, idx_start: int) -> pd.DataFrame:

    # Se unen las líneas desde idx_start hasta final de los datos
    block = "\n".join(lines[idx_start:])

    df_raw = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
    )
    df_raw.columns = [str(c).strip() for c in df_raw.columns]
    df_raw = df_raw.reset_index()

    cols = list(df_raw.columns)
    df_raw.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izq

    # Eliminar columnas completamente vacías (NaN en todas las filas)
    df_raw = df_raw.dropna(axis=1, how="all")

    return df_raw

"------------------------------------------------------------------------------"

# FUNCIONES: Extracción de datos estadísticos necesarios para los seguimientos

# Limitar a 3 cifras significativas
def sig(x, n=5):
    try:
        return float(f"{x:.{n}g}")
    except:
        return x

# Construcción de la tabla final para seguimientos
def estadisticos_resumen(df_stats):

    variables = df_stats.columns.tolist()
    filas = []

    for var in variables:
        try:
            # Extraer valores de df_stats
            valor_nominal = df_stats.loc["Valor nominal", var]
            xqq           = df_stats.loc["xqq", var]
            sigma         = df_stats.loc["Sigma", var]
            cp            = df_stats.loc["Cp", var]
            cpd           = df_stats.loc["Cpd", var]

            # Calcular Rango
            rango = df_stats.loc["Tolerancia superior", var] - valor_nominal

            # Calcular Desviación
            desviacion = (cp - cpd) / (cp) if cp != 0 else np.nan

            # Convertir a 3 cifras significativas cada valor de interés
            fila = [
                var,
                sig(rango),
                sig(valor_nominal),
                sig(xqq),
                sig(sigma),
                sig(cp),
                sig(cpd),
                sig(desviacion)
            ]
        # Si se encuentran valores faltantes devuelve toda la fila como "NaN"
        except KeyError:
            fila = [var] + [np.nan]*7

        filas.append(fila)

    # Se crea el DF final
    df_resumen = pd.DataFrame(
        filas,
        columns=[
            "Variables", "Rango", "Valor nominal", "Media",
            "Sigma_W", "Cp", "Cpk", "Desviación"
        ]
    )

    return df_resumen

"------------------------------------------------------------------------------"

# FUNCIONES: Clasificación de datos por subgrupos y cálculos SPC
# De datos crudos se clasifica para n=5 mediciones por subgrupo y se realizan cálculos SPC
def procesar_datos_spc(df_raw):

    df = df_raw.copy()

    # Se separa el subgrupo con la medición en dos columnas aparte
    df[["subgrupo", "pos"]] = df["Muestra aleatoria"].str.split("/", expand=True)
    df["subgrupo"] = df["subgrupo"].astype(int)
    df["pos"] = df["pos"].astype(int)

    # Toma solo las columnas con valores númericos (Medición de cada variable de control)
    columnas_no_variables = ["Muestra aleatoria", "Momento", "Cantidad piezas", "subgrupo", "pos"]
    variables = [c for c in df.columns if c not in columnas_no_variables]

    # Calcular X̄ y S por subgrupo
    df_xbar = df.groupby("subgrupo")[variables].mean()   # Media por subgrupo
    df_s = df.groupby("subgrupo")[variables].std(ddof=1) # Des Std por subgrupo

    xbar_prom_global = df_xbar.mean() # Cálculo de la media de todas las medias
    s_prom_global = df_s.mean()       # Cálculo de la media de las desv std

    return df, df_xbar, df_s, xbar_prom_global, s_prom_global

"------------------------------------------------------------------------------"

# GRÁFICAS

# FUNCIONES: Generación de la carta I por variable controlada
# Grafica con slider, identificación de puntos fuera de tolerancia
# Dropdown para seleccionar la variable de interés
def graficar_carta_i(df_ind, df_stats, variable, window=20, titulo=None):
    # Datos base
    vals_ind = df_ind[variable].astype(float).to_numpy()     # valores individuales
    n_obs = len(vals_ind)
    muestras = np.arange(1, n_obs + 1)                       # índice de muestra 1..N

    # Valor nominal desde df_stats
    valor_nominal = float(df_stats.loc["Valor nominal", variable])

    # Límites de especificación
    USL = float(df_stats.loc["Tolerancia superior", variable])
    LSL = float(df_stats.loc["Tolerancia inferior", variable])

    # Límites de control de individuales
    UCLi = float(df_stats.loc["LISx", variable])
    LCLi = float(df_stats.loc["LIIx", variable])

    # Escala eje Y automática
    all_vals = np.concatenate([vals_ind, [USL, LSL, UCLi, LCLi, valor_nominal]])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Ventana deslizante
    if window > n_obs:
        window = n_obs

    if titulo is None:
        titulo = (f"Carta de Individuales - {variable}   "
                  f"VN={valor_nominal:.3f}   USL={USL:.3f}   LSL={LSL:.3f}")

    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n_obs)
        x_win = muestras[start_idx:end_idx]
        y_win = vals_ind[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # Zonas fuera de especificación (relleno rojo translúcido)
        ax.axhspan(USL, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LSL, facecolor="red", alpha=0.08)

        # Serie de individuales
        ax.plot(
            x_win, y_win,
            marker="o",
            linestyle="-",
            color="blue",
            label="Valor individual"
        )

        # Puntos fuera de especificación
        mask_out = (y_win > USL) | (y_win < LSL)
        ax.scatter(
            x_win[mask_out],
            y_win[mask_out],
            s=50,
            color="red",
            edgecolors="black",
            zorder=5,
            label="Fuera de especificación" if mask_out.any() else None
        )

          # Valor nominal (VERDE)
        ax.axhline(
            y=valor_nominal,
            linestyle="-",
            linewidth=1.2,
            color="green",
            label="Valor nominal"
        )
         # Límites de especificación (rojo continuo)
        ax.axhline(
            y=USL,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="USL"
        )
        ax.axhline(
            y=LSL,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="LSL"
        )

        # Límites de control de individuales (naranja punteado)
        ax.axhline(
            y=UCLi,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="UCLi"
        )
        ax.axhline(
            y=LCLi,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="LCLi"
        )

        # Ejes y título
        ax.set_xlabel("Número de muestra")
        ax.set_ylabel(variable)
        ax.set_title(titulo)

        # Eje X acotado a la ventana actual
        ax.set_xlim(x_win.min() - 0.5, x_win.max() + 0.5)
        ax.set_xticks(x_win)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # Marco exterior negro
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # Leyenda fuera de la gráfica
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),
            ncol=6,
            frameon=False,
            fontsize=9
        )

        plt.tight_layout()
        plt.show()

    # Slider sobre las muestras individuales
    max_start = max(n_obs - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
        )

    interact(plot_window, start_idx=slider)
def selector_carta_i(df_ind, df_stats, window=200):

    variables = [c for c in df_ind.columns
                 if c not in ["Muestra aleatoria", "Momento", "Cantidad piezas", "subgrupo", "pos"]]

    def _plot(variable):
        graficar_carta_i(df_ind, df_stats, variable, window=window)

    interact(_plot, variable=Dropdown(options=variables, description="Variable:"))

"------------------------------------------------------------------------------"

# FUNCIONES: Generación de la carta X̄ por variable controlada

def graficar_carta_x(df_xbar, xbar_prom, df_stats, variable, window=30, titulo=None):

    # Datos base
    xbar_subgrupos = df_xbar[variable]                      # Serie con X̄ por subgrupo
    subgrupos = xbar_subgrupos.index.astype(int).to_numpy() # Índices de subgrupos como enteros

    xbar_global = float(xbar_prom[variable])

    # Límites de especificación (tolerancias)
    USLx = float(df_stats.loc["Tolerancia superior", variable])
    LSLx = float(df_stats.loc["Tolerancia inferior", variable])

    # Límites de control
    UCLx = float(df_stats.loc["LISx", variable])
    LCLx = float(df_stats.loc["LIIx", variable])

    # Escala eje Y automática
    all_vals = np.concatenate([
        xbar_subgrupos.values,
        [USLx, LSLx, UCLx, LCLx, xbar_global]
    ])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Ventana deslizante)
    n = len(subgrupos)
    if window > n:
        window = n

    if titulo is None:
        titulo = (f"Carta X̄ - {variable}   "
                  f"SW={xbar_global:.3f}   "
                  f"USL={USLx:.3f}   LSL={LSLx:.3f}")

# FUNCIÓN: Configuración gráfica
    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n)
        sg = subgrupos[start_idx:end_idx]
        vals = xbar_subgrupos.iloc[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # Zonas fuera de especificación (En rojo translúcido)
        ax.axhspan(USLx, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LSLx, facecolor="red", alpha=0.08)

        # Serie X̄
        ax.plot(
            sg, vals,
            marker="o",
            linestyle="-",
            color="blue",
            label="X̄ por subgrupo"
        )

        # Puntos fuera de especificación
        mask_out = (vals > USLx) | (vals < LSLx)
        ax.scatter(
            sg[mask_out],
            vals[mask_out],
            s=50,
            color="red",
            edgecolors="black",
            zorder=5,
            label="Fuera de especificación" if mask_out.any() else None
        )

        # Media global (verde, punteada)
        ax.axhline(
            y=xbar_global,
            linestyle="--",
            linewidth=0.8,
            color="green",
            label="X̄ global"
        )

        # Límites de especificación (rojo continuo)
        ax.axhline(
            y=USLx,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="USLx"
        )
        ax.axhline(
            y=LSLx,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="LSLx"
        )

        # Límites de control (naranja punteado)
        ax.axhline(
            y=UCLx,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="UCLx"
        )
        ax.axhline(
            y=LCLx,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="LCLx"
        )

        # Ejes y título
        ax.set_xlabel("Subgrupos")
        ax.set_ylabel(variable)
        ax.set_title(titulo)

        # Eje X acotado a la ventana actual
        ax.set_xlim(sg.min() - 0.5, sg.max() + 0.5)
        ax.set_xticks(sg)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # Marco exterior negro
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # Leyenda fuera de la gráfica
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),
            ncol=6,
            frameon=False,
            fontsize=9
        )

        plt.tight_layout()
        plt.show()

    # Slider para elegir la ventana de subgrupos
    max_start = max(n - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
    )

    interact(plot_window, start_idx=slider)

# FUNCIÓN: Dropdown para selección de la variable de interés
def selector_carta_x(df_xbar, xbar_prom, df_stats, window=30):
    # Lista de variables disponibles (columnas de df_xbar)
    variables = list(df_xbar.columns)

    def _plot(variable):
        graficar_carta_x(df_xbar, xbar_prom, df_stats, variable, window=window)

    interact(_plot, variable=Dropdown(options=variables, description="Variable:"))

"------------------------------------------------------------------------------"

# FUNCIONES: Generación de la carta S por variable controlada
def graficar_carta_s(df_s, s_prom, df_stats, variable, window=30, titulo=None):
    # Datos base
    s_subgrupos = df_s[variable]                          # Serie S por subgrupo
    subgrupos = s_subgrupos.index.astype(int).to_numpy()  # Índices de subgrupos como enteros

    s_global = float(s_prom[variable])

    # Límites de control de S
    UCLs = float(df_stats.loc["LISs", variable])   # Límite superior de control de S
    LCLs = float(df_stats.loc["LIIs", variable])   # Límite inferior de control de S

    # Escala eje Y automática
    all_vals = np.concatenate([
        s_subgrupos.values,
        [UCLs, LCLs, s_global]
    ])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Ventana deslizante
    n = len(subgrupos)
    if window > n:
        window = n

    if titulo is None:
        titulo = (f"Carta S - {variable}   "
                  f"S̄={s_global:.3f}   UCLs={UCLs:.3f}   LCLs={LCLs:.3f}")

    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n)
        sg = subgrupos[start_idx:end_idx]
        vals = s_subgrupos.iloc[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # Zonas fuera de control
        ax.axhspan(UCLs, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LCLs, facecolor="red", alpha=0.08)

        # Serie S
        ax.plot(
            sg, vals,
            marker="o",
            linestyle="-",
            color="blue",
            label="S por subgrupo"
        )

        # Puntos fuera de control (fuera de UCLs / LCLs)
        mask_out = (vals > UCLs) | (vals < LCLs)
        ax.scatter(
            sg[mask_out],
            vals[mask_out],
            s=50,
            color="red",
            edgecolors="black",
            zorder=5,
            label="Fuera de control" if mask_out.any() else None
        )

        # Media global de S (verde, punteada fina)
        ax.axhline(
            y=s_global,
            linestyle="--",
            linewidth=0.8,
            color="green",
            label="S̄ global"
        )

        # Límites de control (naranja punteado)
        ax.axhline(
            y=UCLs,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="UCLs"
        )
        ax.axhline(
            y=LCLs,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="LCLs"
        )

        # Ejes y título
        ax.set_xlabel("Subgrupos")
        ax.set_ylabel(f"S de {variable}")
        ax.set_title(titulo)

        # Eje X acotado a la ventana actual
        ax.set_xlim(sg.min() - 0.5, sg.max() + 0.5)
        ax.set_xticks(sg)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # Marco exterior negro
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # Leyenda fuera de la gráfica
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),
            ncol=4,
            frameon=False,
            fontsize=9
        )

        plt.tight_layout()
        plt.show()

    # Slider para elegir la ventana de subgrupos
    max_start = max(n - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
    )

    interact(plot_window, start_idx=slider)


def selector_carta_s(df_s, s_prom, df_stats, window=30):
    variables = list(df_s.columns)

    def _plot(variable):
        graficar_carta_s(df_s, s_prom, df_stats, variable, window=window)

    interact(_plot, variable=Dropdown(options=variables, description="Variable:"))